In [15]:
import cv2
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import torch.nn.functional as F

import os

In [16]:
class LightweightCNN(nn.Module):
    def __init__(self):
        super(LightweightCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=4, stride=2)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(32, 6)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool2(x)
        x = F.relu(self.conv3(x))
        x = self.maxpool3(x)

        # Reshape for fully connected layer
        x = torch.flatten(x, 1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [17]:
# Load an image from path and convert it to a tensor
def load_image(path):
    img = cv2.imread(path)
    
    # Resize image to 160x120 using INTER_AREA interpolation
    img = cv2.resize(img, (160, 120), interpolation=cv2.INTER_AREA)

    # Convert image to RGB and normalize
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
    cv2.normalize(img_rgb, img_rgb, 0, 1, cv2.NORM_MINMAX)

    # Swap the dimensions and convert to tensor
    img_tensor = torch.as_tensor(img_rgb).permute(2, 0, 1)

    return img_tensor.unsqueeze(0)

# Load the model from path
def load_model(path):
    model = torch.load(path)

    return model

def make_prediction(model, img_tensor):
    model.eval()
    with torch.no_grad():
        # Run the model
        out = model(img_tensor)

        #print(out)

        # Retrieve the index of the highest activation
        _, pred = torch.max(out, 1)

        if pred == 0:
            print("Green & Fallen")
        elif pred == 1:
            print("Green & Standing")
        elif pred == 2:
            print("Red & Fallen")
        elif pred == 3:
            print("Red & Standing")
        elif pred == 4:
            print("White & Fallen")
        elif pred == 5:
            print("White & Standing")
        else:
            print("Error")

In [18]:
# Load the model
model = LightweightCNN()
model.load_state_dict(torch.load("/Users/jvelasquez/Virginia_Tech/Spring_2023/ECE_4806/ieee-robotics-2023-code/src/jetson_code/pedestal_classification/lightweight_net_color_orientation_v3.pth"))

<All keys matched successfully>

In [19]:
# Load the image
path = "/Users/jvelasquez/Downloads/pond/upright/"

# Go through all pictures in the path and make predictions
for filename in os.listdir(path):
    if filename.endswith(".jpg"):
        img_tensor = load_image(path + filename)
        print("Prediction for " + filename + ":")
        make_prediction(model, img_tensor)
        continue
    else:
        continue


Prediction for nvcamtest_8892_s00_00054.jpg:
Green & Standing
Prediction for nvcamtest_8892_s00_00040.jpg:
Green & Standing
Prediction for nvcamtest_8892_s00_00068.jpg:
Green & Standing
Prediction for nvcamtest_8892_s00_00097.jpg:
Green & Standing
Prediction for nvcamtest_8892_s00_00083.jpg:
Green & Standing
Prediction for nvcamtest_8275_s00_00058.jpg:
White & Fallen
Prediction for nvcamtest_8275_s00_00064.jpg:
White & Standing
Prediction for nvcamtest_8275_s00_00070.jpg:
White & Standing
Prediction for nvcamtest_8275_s00_00071.jpg:
White & Standing
Prediction for nvcamtest_8275_s00_00065.jpg:
White & Standing
Prediction for nvcamtest_8275_s00_00059.jpg:
White & Fallen
Prediction for nvcamtest_8892_s00_00082.jpg:
Green & Standing
Prediction for nvcamtest_8892_s00_00096.jpg:
Green & Standing
Prediction for nvcamtest_8892_s00_00069.jpg:
Green & Standing
Prediction for nvcamtest_8892_s00_00041.jpg:
Green & Standing
Prediction for nvcamtest_8892_s00_00055.jpg:
Green & Standing
Prediction f